## Basic tests

In [5]:

from ruamel.yaml import StringIO
from six import StringIO

list1= ["Select asdasl"]
list2= "Select dasahdka"
list3= ["asdasda","asdasda"]


print(type(list1))
print(type(list2))
print(type(list3))

<class 'list'>
<class 'str'>
<class 'list'>


In [6]:
result = dc.query("SELECT count(column_name) as column_count FROM information_schema.columns WHERE table_name = 'test_table' and column_name != 'id'")[0][0]
print(f"--- {result}")

--- 0


In [7]:
result = dc.query("SELECT column_name as column_count FROM information_schema.columns WHERE table_name = 'test_table' and column_name != 'id'")
print(f">\n {result}")

list = tuple([column[0] for column in result])
print(list)

print(f"INSERT INTO table {list}")

>
 []
()
INSERT INTO table ()


In [8]:
return_value = [('name',), ('bezeichnung',)]
count = len(return_value)
print(count)

new_list = []
for i in return_value:
    new_list.append(i[0])

print(new_list)

new_list2 = [column[0] for column in return_value]
print(new_list2)

filtered = len(return_value)
print(f"({('%s,'*(filtered-1) + '%s')})")


2
['name', 'bezeichnung']
['name', 'bezeichnung']
(%s,%s)


In [9]:
values1 = None
values = ()

def ubs():
    if not values or len(values) < 1:
        return ("Please provide a 'value' when using this function")

ubs()

"Please provide a 'value' when using this function"

In [10]:
def generate_create_statement(table: str, values: tuple[tuple[str, str], ...]) -> str:
    columns = []
    for column_name, column_type in values:
        columns.append(f"{column_name} {column_type}")

    columns_sql = ",\n\t".join(columns)
    statement = f"CREATE TABLE {table} (\n\t{columns_sql}\n);"
    return statement


table = "ernst"
values = (
    ("id", "SERIAL PRIMARY KEY"),
    ("name", "varchar(255)"),
    ("type", "varchar(100)")
)

print(generate_create_statement(table,values))

CREATE TABLE ernst (
	id SERIAL PRIMARY KEY,
	name varchar(255),
	type varchar(100)
);


## Database tests
------------------------

In [8]:
import cloud_data.connectors.database_connector as dc

#### Query

In [10]:
#statement = "SELECT * FROM test"
statement = f"SELECT * FROM staging_gamedata WHERE processed=True"
print(dc.query(statement))

[(403, 'zFaPge', 'Man', '4', '1.38', True, 50, 400, 715, 0, 0), (404, 'zFaPge', 'Man', '4', '1.54', True, 50, 400, 715, 0, 0), (405, 'zFaPge', 'Man', '4', '1.72', True, 50, 400, 715, 0, 0), (406, 'zFaPge', 'Man', '1', '1.87', True, 50, 400, 715, 0, 0), (407, 'zFaPge', 'Man', '4', '1.9', True, 50, 386, 715, 0, 0), (408, 'zFaPge', 'Man', '2', '2.27', True, 50, 204, 715, 0, 0), (409, 'zFaPge', 'Man', '4', '2.37', True, 50, 218, 715, 0, 0), (410, 'zFaPge', 'Man', '4', '2.53', True, 46, 253, 715, 0, 0), (411, 'zFaPge', 'Man', '1', '2.56', True, 44, 253, 715, 0, 0), (412, 'zFaPge', 'Man', '3', '2.62', True, 40, 225, 715, 0, 0), (413, 'zFaPge', 'Man', '2', '2.67', True, 37, 204, 659, 0, 0), (414, 'zFaPge', 'Man', '4', '2.78', True, 37, 246, 547, 0, 0), (415, 'zFaPge', 'Man', '4', '2.99', True, 37, 288, 475, 0, 0), (416, 'zFaPge', 'Man', '4', '3.17', True, 37, 288, 547, 0, 0), (417, 'zFaPge', 'Man', '4', '3.33', True, 37, 288, 715, 0, 0), (418, 'zFaPge', 'Man', '2', '3.38', True, 37, 288, 715,

#### Check Columns

In [3]:
tables = ["test","staging"]
dc.check_columns(tables[1])

[('col1',),
 ('col2',),
 ('col3',),
 ('col4',),
 ('col5',),
 ('col6',),
 ('col7',),
 ('col8',),
 ('col9',),
 ('processed',)]

In [28]:
data4 = [('Ivan',30),('Thomas',34),('Stefan',40)]
data5 = [('Ivan',30)]

print(data5[0])

('Ivan', 30)


#### Insert

##### .execute()

In [4]:
#Test mit .execute()

table = ["test","staging"]
values = [('Ivan',23), ("working","testscore",18,27,"Herbert","","","","",True),("faulty","testscore",17,25,"Herbert","","","",True)]
values_listoftuple = [("working","testscore3",18,27,"Herbert","","","","",True),("working","testscore2",18,27,"Herbert","","","","",True)]
# Table test hat die Columns "name", "number" ist alphabetisch
# Table staging id, col1-col15,processed


print(dc.insert(table=table[1],amount="single",values=values[1]))

INSERT INTO staging (col1, col2, col3, col4, col5, col6, col7, col8, col9, processed) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
INSERT 0 1


##### .executemany()

In [5]:
#Test mit .executemany()

table = "test"
# Table test hat die Columns "name", "number" ist alphabetisch
manyvalues = [('Ivan',30),('Thomas',34),('Stefan',40)]
print(dc.insert(table=table,amount="many",values=manyvalues))

INSERT INTO test (name, number) VALUES (%s,%s)
INSERT 0 1


#### COPY

In [3]:
filepath = ["airflow_data/testdb.csv","airflow_data/dummy_file.csv.csv"]
tablename = ["test","staging_gamedata"]
columns = [("name","number"),("game_id","user_name","user_input","timestamp")]
dc.copy_to_db(filepath[1],tablename[1],columns[1],format="CSV",header=True,delimiter=",")

File (airflow_data/dummy_file.csv.csv) copied succesfully to table staging_gamedata.


In [5]:
tablecolumns = dc._placeholder(dc.check_columns('staging_bbox'))
#print(tablecolumns[0]) # columns_count
#print(tablecolumns[1]) # columns_string
print(tablecolumns[2]) # columns_tuple
#print(tablecolumns[3]) # value_placeholder

filepath = "airflow_data/data_bbox/562db41a-9d87-4570-a84a-e1b5fb880578.csv"
tablename = "staging_bbox"
columns = ['training', 'epoch', 'current_time_start', 'current_time_end', 'mean_squared_error', 'mean_center_dist', 'mean_size_error', 'mean_overlap', 'combined_score', 'acc_all_conditions', 'train_loss', 'val_loss', 'train_acc', 'val_acc', 'current_lr', 'IMG_SIZE', 'BATCH_SIZE']
dc.copy_to_db(filepath,tablename,columns,format="CSV",header=True,delimiter=",")



('training', 'epoch', 'current_time_start', 'current_time_end', 'mean_squared_error', 'mean_center_dist', 'mean_size_error', 'mean_overlap', 'combined_score', 'acc_all_conditions', 'train_loss', 'val_loss', 'train_acc', 'val_acc', 'current_lr', 'img_size', 'batch_size')
File (airflow_data/562db41a-9d87-4570-a84a-e1b5fb880578.csv) copied succesfully to table staging_bbox.


In [3]:
dc.check_columns('staging_gamedata')

[('game_id',), ('user_name',), ('user_input',), ('timestamp',)]

### Update

In [2]:
table = ["staging_gamedata"]
column = ["game_id"]
value = ["36B0WC"]
action = ["processed"]
dc.update(table[0],column[0],value[0],action=action[0])

'UPDATE 50'

### Delete

In [2]:
table = "staging_gamedata"
statement = f"DELETE FROM {table} WHERE processed = True and id = 37 RETURNING *"

print(dc.insert_manual(statement))


[(37, '36B0WC', 'Max', '2', '10.81', True)]


## Google Cloud Storage (Bucket)
--------------

### Initialize cloud_connector

In [1]:

from cloud_data.connectors import cloud_connector as cc

### Upload

In [4]:
# Full Filepath with filename
cc.upload_to_bucket("airflow_data/testdb.csv")

File airflow_data/testdb.csv uploaded as testdb.csv.


### Download

In [ ]:
# Filename in Bucket and Filelocation
cc.download_from_bucket("test25.txt","test25.txt")

## Airflow
---

In [1]:
import requests
from requests.auth import HTTPBasicAuth

username = "airflow"
password = "airflow"
airflow_endpoint = "http://localhost:8080/api/v1/"

#### Check Version

In [2]:
service = "version"
ENDPOINT_URL=airflow_endpoint + service
response = requests.get(ENDPOINT_URL)
print(response.json())

{'git_version': '.release:2.0.2+e494306fb01f3a026e7e2832ca94902e96b526fa', 'version': '2.0.2'}


#### List DAGs

In [3]:
service = "dags"
ENDPOINT_URL=airflow_endpoint + service

response = requests.get(ENDPOINT_URL,auth=HTTPBasicAuth(username, password))
if response.status_code == 200:
    print(response.json())
    print(response.json()['dags'][0]["dag_id"])
    dags_id = response.json()['dags'][0]["dag_id"]

else:
    print(response.status_code)
    print(response.text)

{'dags': [{'dag_id': 'CSV_Import_and_Processing_Pipeline', 'description': None, 'file_token': 'Ii9vcHQvYWlyZmxvdy9kYWdzL3BpcGVsaW5lX2Nzdl9tYWluLnB5Ig.UA84HMmf6BMY4vQI0T6zGHuAgJI', 'fileloc': '/opt/airflow/dags/pipeline_csv_main.py', 'is_paused': True, 'is_subdag': False, 'owners': ['airflow'], 'root_dag_id': None, 'schedule_interval': {'__type': 'CronExpression', 'value': '5 4 * * sun'}, 'tags': []}, {'dag_id': 'Delete_processed', 'description': None, 'file_token': 'Ii9vcHQvYWlyZmxvdy9kYWdzL3BpcGVsaW5lX2RlbGV0ZS5weSI.Xa3cJIM9Mz5HjBLcIlLbAqLMzHM', 'fileloc': '/opt/airflow/dags/pipeline_delete.py', 'is_paused': False, 'is_subdag': False, 'owners': ['airflow'], 'root_dag_id': None, 'schedule_interval': {'__type': 'CronExpression', 'value': '5 4 * * sun'}, 'tags': []}, {'dag_id': 'Process_Gaming_Staging_Data', 'description': None, 'file_token': 'Ii9vcHQvYWlyZmxvdy9kYWdzL3BpcGVsaW5lX3Byb2Nlc3Nfc3RhZ2luZy5weSI.jHBgzQ3Rh44iz-KGOgQKCnxtfOs', 'fileloc': '/opt/airflow/dags/pipeline_process_stagi

#### Information about DAG

In [5]:
service = f"dags/{dags_id}"
ENDPOINT_URL=airflow_endpoint + service

response = requests.get(ENDPOINT_URL,auth=HTTPBasicAuth(username, password))
if response.status_code == 200:
    print(response.json())
else:
    print(response.status_code)
    print(response.text)

{'dag_id': 'CSV_Import_and_Processing_Pipeline', 'description': None, 'file_token': 'Ii9vcHQvYWlyZmxvdy9kYWdzL3BpcGVsaW5lX2Nzdl9tYWluLnB5Ig.UA84HMmf6BMY4vQI0T6zGHuAgJI', 'fileloc': '/opt/airflow/dags/pipeline_csv_main.py', 'is_paused': True, 'is_subdag': False, 'owners': ['airflow'], 'root_dag_id': None, 'schedule_interval': {'__type': 'CronExpression', 'value': '5 4 * * sun'}, 'tags': []}


#### Start DAG

In [7]:
import json

# Run "List DAGs" to obtain first dags_id

service = f"dags/{dags_id}/dagRuns"
ENDPOINT_URL=airflow_endpoint + service

headers = {
    "Content-Type": "application/json"
}

data = {
    "conf": {}
}

response = requests.post(ENDPOINT_URL, headers=headers, data=json.dumps(data),auth=(username, password))

# Ergebnis anzeigen
if response.status_code == 200:
    print("DAG started successfully:", response.json())
else:
    print("Error when starting DAGs:", response.status_code, response.text)

DAG wurde erfolgreich gestartet: {'conf': {}, 'dag_id': 'CSV_Import_and_Processing_Pipeline', 'dag_run_id': 'manual__2025-06-20T08:02:36.811848+00:00', 'end_date': None, 'execution_date': '2025-06-20T08:02:36.811848+00:00', 'external_trigger': True, 'start_date': '2025-06-20T08:02:36.815637+00:00', 'state': 'running'}
